# Analyse clusters of international declarations

In [1]:
# Imports
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load log
import sys
sys.path.insert(1, '../')
from src.io import PRE, read_log

ptc_log = read_log(PRE)

In [3]:
def analyze_request(request):
    no_submissions = 0
    reimbursed = False
    

    for event in request:
        activity_type = event["concept:name"].lower()
        if activity_type == "request for payment submitted by employee":
            no_submissions += 1
        elif activity_type == "payment handled":
            reimbursed = True

    return (no_submissions, reimbursed)


def extract_info_from_request(request):
    (no_submissions, reimbursed) = analyze_request(request)
    
    attributes = request[0]

    decl_obj = {
        'id': attributes["(case)_Rfp_id"],
        'amount': float(attributes["(case)_RequestedAmount"]),
        'permit_budget': attributes["(case)_Permit_BudgetNumber"],
        'org': attributes["(case)_OrganizationalEntity"],
        'permit_org': attributes["(case)_Permit_OrganizationalEntity"],
        'project': attributes["(case)_Project"],
        'permit_project': attributes["(case)_Permit_ProjectNumber"],
        'requested_budget': float(attributes["(case)_Permit_RequestedBudget"]),
        'permit': attributes["(case)_Permit_id"],
        'submissions': no_submissions,
        'reimbursed': reimbursed
    }
    return decl_obj

In [4]:
# Schema: [{'id', 'amount', 'budget', 'permit_budget', 'activity', 'org', 'project', 'requested_budget', 'task', 'permit', 'travel_time', 'submissions', 'reimbursed'}]
requests = []

for request in ptc_log:
    
    requests.append(extract_info_from_request(request))

ptc_dataframe = pd.DataFrame(requests)
ptc_dataframe.set_index('id', drop=False)


,id,amount,permit_budget,org,permit_org,project,permit_project,requested_budget,permit,submissions,reimbursed
id,,,,,,,,,,,
request for payment 73550,request for payment 73550,854.579838,budget 6198,organizational unit 65463,organizational unit 65455,project 503,UNKNOWN,1979.272104,travel permit 73549,1,False
request for payment 73552,request for payment 73552,854.579838,budget 6198,organizational unit 65463,organizational unit 65455,project 503,UNKNOWN,1979.272104,travel permit 73549,1,True
request for payment 76316,request for payment 76316,1173.957795,budget 2233,organizational unit 65463,organizational unit 65455,project 503,UNKNOWN,3553.601973,travel permit 76314,1,True
request for payment 73536,request for payment 73536,790.552073,budget 899,organizational unit 65463,organizational unit 65455,project 503,project 2260,3627.364438,travel permit 73535,1,True
request for payment 76195,request for payment 76195,181.978003,budget 4164,organizational unit 65463,organizational unit 65455,project 503,UNKNOWN,265.661318,travel permit 76193,1,True
...,...,...,...,...,...,...,...,...,...,...,...
request for payment 186317,request for payment 186317,251.008620,UNKNOWN,organizational unit 65456,UNKNOWN,project 151278,UNKNOWN,0.000000,UNKNOWN,1,True
request for payment 186241,request for payment 186241,261.459608,UNKNOWN,organizational unit 65468,UNKNOWN,project 147556,UNKNOWN,0.000000,UNKNOWN,1,True
request for payment 48748,request for payment 48748,1511.596873,budget 1755,organizational unit 65469,organizational unit 65454,project 503,project 48747,1827.310143,travel permit 48745,1,False


In [5]:
ptc_dataframe['reimbursed'].mean()

0.9511709018435476

In [6]:

org_group = ptc_dataframe.groupby(['org'])
org_group.agg({
    'amount': 'mean',
    'submissions': 'mean',
    'reimbursed': ['mean', 'size']
    }).sort_values(by=('reimbursed', 'size'), ascending=False)

amount submissions reimbursed     
                                   mean        mean       mean size
org                                                                
organizational unit 65461    686.843526    1.125382   0.957187  327
organizational unit 65463    761.462500    1.020202   0.942761  297
organizational unit 65458    856.013544    1.027778   0.958333  288
organizational unit 65454    621.400451    1.104478   0.973881  268
organizational unit 65469    749.675184    1.106481   0.953704  216
organizational unit 65462    835.325322    1.111111   0.937500  144
organizational unit 65456    514.093513    1.117117   0.981982  111
organizational unit 65465    611.221295    1.049020   0.901961  102
organizational unit 65468  18526.409722    1.073171   0.926829   82
organizational unit 65457    787.445716    1.189655   0.965517   58
organizational unit 65475    762.577356    1.270270   0.972973   37
organizational unit 65476    854.864336    1.125000   0.968750   32
organizational unit 65482    354.144773    1.250000   0.875000    8
organizational unit 65466    427.626501    1.166667   0.666667    6
organizational unit 65472    641.595788    1.166667   0.833333    6
organizational unit 65478   1421.220678    1.000000   1.000000    5
organizational unit 65481   1530.119633    1.000000   1.000000    5
organizational unit 65487   1216.250736    1.000000   0.666667    3
organizational unit 65488    230.271423    1.000000   0.666667    3
organizational unit 65480    793.141312    1.000000   1.000000    2
organizational unit 65477    480.901917    1.000000   1.000000    2
organizational unit 65455    458.777882    1.000000   1.000000    2
organizational unit 65473     67.744147    1.000000   0.000000    1
organizational unit 65485    614.325020    1.000000   1.000000    1
organizational unit 65471   1128.244476    1.000000   1.000000    1

In [7]:
project_group = ptc_dataframe.groupby(['project'])
project_group.agg({
    'amount': 'mean',
    'submissions': 'mean',
    'reimbursed': ['mean', 'size']
    }).sort_values(by=('reimbursed', 'size'), ascending=False)

amount submissions reimbursed      
                       mean        mean       mean  size
project                                                 
project 503     1520.142604    1.092593   0.962963  1890
UNKNOWN          591.306033    0.814815   0.000000    27
project 147556   257.043428    1.066667   1.000000    15
project 147546   136.153187    1.000000   1.000000     8
project 148052  1004.788680    1.125000   1.000000     8
project 147620   410.579192    1.125000   1.000000     8
project 147531   306.597268    1.000000   1.000000     7
project 151278   393.610495    1.000000   1.000000     6
project 147582   117.894452    1.000000   0.800000     5
project 147649   102.763809    1.000000   1.000000     4
project 147860   453.654805    1.000000   1.000000     4
project 147551   197.213643    1.000000   1.000000     3
project 148681   117.597460    1.000000   1.000000     3
project 147843   191.436368    1.500000   1.000000     2
project 147701    17.763360    1.000000   1.000000     2
project 150968    72.740798    1.000000   1.000000     2
project 147572    35.762189    1.000000   1.000000     2
project 152803   323.389755    1.000000   1.000000     2
project 155217  1050.440171    1.500000   1.000000     2
project 148982   214.216302    1.000000   1.000000     1
project 150197   139.390248    1.000000   1.000000     1
project 151211    78.304584    1.000000   1.000000     1
project 155098   111.073008    1.000000   1.000000     1
project 156832  1651.726820    1.000000   1.000000     1
project 164433   306.475343    1.000000   1.000000     1
project 148309    19.960877    1.000000   1.000000     1